<a href="https://colab.research.google.com/github/Tomawock/NLP_Attack/blob/main/utils/Amazon_reduceAndChunk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install textattack

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
from textattack.augmentation import WordNetAugmenter
from textattack.augmentation import EmbeddingAugmenter

In [8]:
dataset = pd.read_csv('/content/drive/Shareddrives/Deep Learning/datasets/Amazon/Reviews.csv')
print(f"COLONNE:{dataset.columns}")
print(f"DIMENSIONE:{dataset.shape}")

COLONNE:Index(['Id', 'ProductId', 'UserId', 'ProfileName', 'HelpfulnessNumerator',
       'HelpfulnessDenominator', 'Score', 'Time', 'Summary', 'Text'],
      dtype='object')
DIMENSIONE:(568454, 10)
COLONNE:Index(['Score', 'Summary', 'Text'], dtype='object')
DIMENSIONE:(393579, 3)


##Filtering del dataset dei dati duplicati e rimozione parti inutili di esso

In [ ]:
rar=dataset.copy()
rar.drop_duplicates(subset ="Text",
                    keep = "last", inplace = True)
rar.drop(['Id','ProductId', 'UserId', 'ProfileName', 'HelpfulnessNumerator','HelpfulnessDenominator','Time'], axis=1, inplace=True)
print(f"COLONNE:{rar.columns}")
print(f"DIMENSIONE:{rar.shape}")

In [11]:
rar["review_type"] = rar["Score"].apply(lambda x: "negative" if x < 4 else "positive")

In [12]:
rar.review_type.value_counts()

positive    306756
negative     86823
Name: review_type, dtype: int64

In [14]:
positive_reviews = rar[rar.review_type == "positive"]
negative_reviews = rar[rar.review_type == "negative"]

In [31]:
positive_df = positive_reviews.sample(n=len(negative_reviews), random_state=22)
negative_df = negative_reviews

In [56]:
df = pd.concat(objs=[positive_df, negative_df]).sample(frac=1).reset_index(drop=True)
df

,Score,Summary,Text,review_type
0,5,excellent steak and meat seasoning,The Lynchburg seasoning adds great flavor to a...,positive
1,3,Just Okay,I was very happy to find this item at Amazon. ...,negative
2,5,"Awesome, Who do I tell?!",I just had two of these heated in the microwav...,positive
3,1,Most weren't flavored,I didn't expect to receive the exact mix that ...,negative
4,4,McCann's Steel Cut Oatmeal,What a treat! This oatmeal is fantastic and al...,positive
...,...,...,...,...
173641,2,Ok but tastes suspiciously like Campbell's Select,"This soup is ok, but it's certainly not what y...",negative
173642,1,not my cup of coffee,i bought this becuase i liked the nescaf&eacut...,negative
173643,5,Amazing coffee!,I absolutely love this coffee and even though ...,positive
173644,1,BUGS IN THE CHOCOLATE,I FOUND BUGS IN THE PACKETTE! IF YOU DONT LIKE...,negative


In [63]:
df.to_csv('/content/drive/Shareddrives/Deep Learning/datasets/Amazon/reducedReviews.csv', index=False)

In [67]:
df.iloc[[i*2000 for i in range(10)]]

,Score,Summary,Text,review_type
0,5,excellent steak and meat seasoning,The Lynchburg seasoning adds great flavor to a...,positive
2000,2,"Edible, But Not as Good As Hoped For",I must start off by saying that I really reall...,negative
4000,5,Salon Quality,I was impressed with this shampoo. It has a n...,positive
6000,1,Taste just like real pasta,Because it is. I am a low carb eater and one s...,negative
8000,2,terminally mediocre (in my hands),My experience is quite different to most other...,negative
10000,5,BROWN RICE CAKES,These are absolutely the best rice cakes in th...,positive
12000,1,WAY TOO MUCH SEASONING,I am sorry I bought a dozen. I though I could ...,negative
14000,5,The best cereal!,My family loves this cereal! It is the only k...,positive
16000,5,Outstanding gingerbread flavor!,Went to a Red Robin's chain restaurant the oth...,positive
18000,5,Peachy Goodness,I purchased this item To use as wedding favors...,positive


#### Function to chunk

In [64]:
def chunk(dataset, dim=2000, max_iteration=60):
  for i in range(min(len(dataset)//dim, max_iteration)):
    yield dataset[i*dim:(i+1)*dim]

### Parallel execution

In [92]:
import threading

In [89]:
df.iloc[[2000*i for i in range(10)], [0]]

,Score
0,5
2000,2
4000,5
6000,1
8000,2
10000,5
12000,1
14000,5
16000,5
18000,5


In [100]:
def stampa(subset):
  print(subset.iloc[0])

In [101]:
i = 0
for subset in chunk(df):
  threading.Thread(target=stampa,
                   args=[subset]).start()

Score                                                          5
Summary                       excellent steak and meat seasoning
Text           The Lynchburg seasoning adds great flavor to a...
review_type                                             positive
Name: 0, dtype: object
Score                                                          2
Summary                     Edible, But Not as Good As Hoped For
Text           I must start off by saying that I really reall...
review_type                                             negative
Name: 2000, dtype: object
Score                                                          5
Summary                                            Salon Quality
Text           I was impressed with this shampoo.  It has a n...
review_type                                             positive
Name: 4000, dtype: object
Score                                                          1
Summary                               Taste just like real pasta
Text           